# Convolution Thermal

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

In [ ]:
from materialite.models.convolution_model import ConvolutionModel, LaserBeam
from materialite import Material
import numpy as np

Materialite implements a method currently termed the 'convolution method' for simulating the temperature fields resulting from laser heating. To use this model, first a laser path needs to be created:

In [ ]:
material = Material(dimensions=[200, 60, 40], spacing=[5e-6, 5e-6, 5e-6])
material = material.create_uniform_field("temperature", 300.0)
laser = LaserBeam(
    material,
    laser_power=100,
    hatch_spacing=20,
    beam_x_radius=50.0e-6,
    beam_y_radius=50.0e-6,
    beam_z_radius=30.0e-6,
    num_layers=1,
    scan_offset_start=-10,
    hatch_offset_start=30,
    hatch_offset_end=-30,
    z_start=40,
    layer_thickness=8,
    time_between_scans=0.000,
    time_between_layers=0.0,
    rotation_angle_increment=np.deg2rad(67),
)

laser.create_build_path(material, adjust_domain=True, domain_epsilon=5e-6)
laser.plot_beam_path()

Next, the model needs to be set up. It requires thermophysical properties to be passed in. The 'enthalpy_props' are enthalpy-related thermophysical parameters that allow for simulating phase change using the enthalpy method. 

In [ ]:
density = 4200.0
specific_heat_capacity = 656.2
thermal_conductivity = 18.72
volumetric_specific_heat = specific_heat_capacity * density
thermal_diffusivity = thermal_conductivity / volumetric_specific_heat

thermophysical_props = {
    "thermal_diffusivity": thermal_diffusivity,
    "volumetric_specific_heat": volumetric_specific_heat,
    "power_absorptivity": 0.48,
    "melt_temperature": 1928.0,
}

enthalpy_props = {
    "vapor_temperature": 3000.0,
    "heat_of_fusion": 370000.0 * density,
    "heat_of_vapor": 10000000 * density,
    "melt_range": 25.0,
    "vapor_range": 10.0,
}

model = ConvolutionModel(
    save_frequency=50,
    time_steps=175,
    thermophysical_props=thermophysical_props,
    enthalpy_props=enthalpy_props,
    adjust_xy_radii_multiple=3,
    adjust_xy_extra_voxels=0,
    z_range=60,
)

The model can be ran, and the temperature field can be plotted: 

In [ ]:
new_material = model.run(material, laser, "temperature")
new_material.plot("temperature", color_lims=[300, 1500])

The enthalpy method allows for the phase to be predicted as well. The integer values correspond to 0=solid, 1=mushy/partially molten, 2=liquid, and 3=partially vaporized

In [ ]:
new_material.plot("phase")